In [1]:
import numpy as np
# read data from reviews and labels file.
with open('Data.csv', 'r') as f:
    all_accents = f.readlines() #len = 1679154
data_len = len(all_accents)
#!git clone https://git.opendfki.de/annvix/Accents_LSTM-.git

In [2]:
words, accents = [], []

In [3]:
for i in range(data_len):
    str = all_accents[i].replace('\n', '').split(',')
    words.append(str[0])
    accents.append(str[1])
max_len = len(max(words, key=len))

In [4]:
print('Max word length:')
print(max_len)

Max word length:
30


In [5]:
words_int = np.zeros((data_len,max_len), dtype=int)
accents_int = np.zeros(data_len, dtype=int)

In [6]:
for i in range(data_len):
    for j in range(len(words[i])):
        words_int[i][j]=ord(words[i][j])
    accents_int[i] = accents[i].find(chr(39)) + 1

In [7]:
print('Words without accents:')
for i in range(len(words)):
    if accents_int[i]==-1:
        print(words[i])

Words without accents:


In [8]:
train_frac = 0.8
val_frac = 0.1
test_frac = 0.1


def train_test_val_split(words_int):
    return (words_int[:int(len(words_int)*train_frac)], 
words_int[int(len(words_int)*train_frac) : (int(len(words_int)*train_frac) + int(len(words_int)*val_frac))], 
words_int[(int(len(words_int)*train_frac) + int(len(words_int)*val_frac)):])
    
def train_test_val_labels(accents_int):
    return (accents_int[:int(len(accents_int)*train_frac)], 
accents_int[int(len(accents_int)*train_frac) : (int(len(accents_int)*train_frac) + int(len(accents_int)*val_frac))], 
accents_int[(int(len(accents_int)*train_frac) + int(len(accents_int)*val_frac)):])

train_x, val_x, test_x = train_test_val_split(words_int)
train_y, val_y, test_y = train_test_val_labels(accents_int)

In [9]:
## print out the shapes of your resultant feature data
print("\t\t\t Sizes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			 Sizes:
Train set: 		(1343301, 30) 
Validation set: 	(167912, 30) 
Test set: 		(167914, 30)


In [10]:
print("\t\t Vizualization:")
train = ''
for i in range(10):
    train = ''
    for j in range(len(train_x[i])):
        train = train + chr(train_x[i][j])
    print("Train set: ",train)

for i in range(10):    
    vval = ''
    for j in range(len(val_x[i])):
        vval = vval + chr(val_x[i][j])
    print("Validation set: ",vval)

for i in range(10):
    test = ''
    for j in range(len(test_x[i])):
        test = test + chr(test_x[i][j])
    print("Test set: ",test)


		 Vizualization:
Train set:  а                             
Train set:  аав                           
Train set:  аава                          
Train set:  аавам                         
Train set:  аавами                        
Train set:  аавасакса                     
Train set:  аавах                         
Train set:  ааве                          
Train set:  аавик                         
Train set:  аавов                         
Validation set:  самоокупитесь                 
Validation set:  самоокупится                  
Validation set:  самоокупиться                 
Validation set:  самоокупишься                 
Validation set:  самоокуплюсь                  
Validation set:  самоокупятся                  
Validation set:  самооплодотворение            
Validation set:  самооплодотворением           
Validation set:  самооплодотворении            
Validation set:  самооплодотворений            
Test set:  трудоустройством              
Test set:  трудоустройству    

In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets for train, test and val
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50 

# make sure to SHUFFLE your training data. Keep Shuffle=True.
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [12]:
print(type(train_data))
print(type(train_x))

<class 'torch.utils.data.dataset.TensorDataset'>
<class 'numpy.ndarray'>


In [13]:
# obtain one batch of training data and label. 
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 30])
Sample input: 
 tensor([[1074, 1099, 1089,  ...,    0,    0,    0],
        [1073, 1072, 1088,  ...,    0,    0,    0],
        [1085, 1072, 1083,  ...,    0,    0,    0],
        ...,
        [1083, 1091, 1075,  ...,    0,    0,    0],
        [1088, 1072, 1079,  ...,    0,    0,    0],
        [1087, 1086, 1076,  ...,    0,    0,    0]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([ 2,  5,  4,  6,  2,  8,  4,  7,  2,  2,  6,  4,  5,  6,  6,  7,  7,  7,
         4, 12,  6,  8,  2,  8,  4,  8,  4,  2,  2,  8,  9,  6,  8,  6,  6,  7,
        10,  6,  7,  5,  7,  8,  8,  7,  5,  7, 10,  4,  7,  7],
       dtype=torch.int32)


In [14]:
# Check if GPU is available.
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [15]:
import torch.nn as nn

class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, data_len, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(data_len, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.relu = nn.ReLU()#Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        relu_out = self.Relu(out)#sig_out = self.sig(out)
        
        # reshape to be batch_size first
        relu_out = relu_out.view(batch_size, -1)
        relu_out = relu_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return relu_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
    

In [16]:
# Instantiate the model with these hyperparameters
data_len = data_len #+ 1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 300
hidden_dim = 256
n_layers = 2

net = SentimentLSTM(data_len, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentLSTM(
  (embedding): Embedding(1679127, 300)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
)


In [17]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
